In [7]:
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

PROJECT_PATH = os.getenv('PROJECT_DIR')
sys.path.append(PROJECT_PATH)

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 6)

In [8]:
from src.data_ingestion.iex_data import IexDataFetcher
from src.data_ingestion.weather_data import WeatherDataFetcher
from src.feature_engineering.build_features import FeatureEngineering
from src.model_building.forecast_model import ModelForecaster
from src.db_insertion.db_insertion import DAMInsertion
from src.utils import *
from config.paths import *

In [9]:
# creating instances
iex_data = IexDataFetcher()
weather_data = WeatherDataFetcher()

featured_data = FeatureEngineering(PROJECT_PATH)
forecasting = ModelForecaster(MODELS_PATH) 
db_insert = DAMInsertion() 

In [10]:
# dam = iex_data._get_processed_data('dam')
# rtm = iex_data._get_processed_data('rtm')
# weather = weather_data._get_processed_weather('weather')
# wind = weather_data._get_processed_weather('wind')
# hydro = weather_data._get_processed_weather('hydro')
# solar = weather_data._get_processed_weather('solar')

In [11]:
dam = load_pickle(PROCESSED_DATA_PATH, 'dam_data')
rtm = load_pickle(PROCESSED_DATA_PATH, 'rtm_data')
weather = load_pickle(PROCESSED_DATA_PATH, 'weather_data')
wind = load_pickle(PROCESSED_DATA_PATH, 'wind_data')
hydro = load_pickle(PROCESSED_DATA_PATH, 'hydro_data')
solar = load_pickle(PROCESSED_DATA_PATH, 'solar_data')

In [12]:
rtm = rtm[rtm['datetime'] < datetime.now().strftime('%Y-%m-%d')]

In [13]:
dam = featured_data.shift_date(dam, 1) 
weather = featured_data.shift_date(weather, 2)
hydro = featured_data.shift_date(hydro, 2) 
solar = featured_data.shift_date(solar, 2) 
wind = featured_data.shift_date(wind, 2)

data = featured_data.merge_dataframes([rtm, dam, weather, hydro, solar, wind])

In [14]:
data = featured_data._get_features(data, weather, market_type = 'rtm', task = 'inference')

In [16]:
forecast_date = forecasting.forecasting_date(data, 'rtm')

In [18]:
forecast = forecasting.create_forecast(data, forecast_date)

KeyError: "['mcp_dam_with_rh_che', 'mcp_dam_with_tb', 'mcp_dam_mean_3d', 'mcp_dam_mean_5d', 'mcp_dam_with_temp_mum', 'mcp_dam_with_doy'] not in index"

In [ ]:
forecast

,datetime,forecast,lower_bound,upper_bound
0,2024-01-17 00:00:00,3404.3,3353.5,3568.1
1,2024-01-17 00:15:00,3375.5,3308.5,3645.1
2,2024-01-17 00:30:00,3401.0,3376.2,3628.2
3,2024-01-17 00:45:00,3389.2,3365.9,3578.0
4,2024-01-17 01:00:00,3238.9,2955.0,3440.9
...,...,...,...,...
91,2024-01-17 22:45:00,NaN,NaN,NaN
92,2024-01-17 23:00:00,NaN,NaN,NaN
93,2024-01-17 23:15:00,NaN,NaN,NaN
94,2024-01-17 23:30:00,NaN,NaN,NaN


In [10]:
db_insert.save_forecast(forecast,  forecast_date, 'forecast')

forecast data not inserted


In [11]:
db_insert.save_forecast(forecast,  forecast_date, 'lower_bound')

lower_bound data inserted successfully


In [12]:
db_insert.save_forecast(forecast,  forecast_date, 'upper_bound')

upper_bound data inserted successfully
